In [ ]:
#| default_exp capture

# Capture Support


:::{.callout-tip}

This module can be imported using `from openhsi.capture import *`

:::


The OpenHSI class defines the interface between custom camera implementations and all the processing and calibration needed to run a pushbroom hyperspectral imager. 

In [ ]:
#| hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

In [ ]:
#| export
#| hide

from fastcore.foundation import patch
from fastcore.meta import delegates
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from PIL import Image
from tqdm import tqdm
import warnings

from typing import Iterable, Union, Callable, List, TypeVar, Generic, Tuple, Optional
import json
import pickle

In [ ]:
#| export

from openhsi.data import DataCube, CircArrayBuffer

In [ ]:
#| export

@delegates()
class OpenHSI(DataCube):
    """Base Class for the OpenHSI Camera."""
    def __init__(self, **kwargs):
        
        super().__init__(**kwargs)
        super().set_processing_lvl(self.proc_lvl)
        if callable(getattr(self,"get_temp",None)):
            self.cam_temperatures = CircArrayBuffer(size=(self.n_lines,),dtype=np.float32)
        
        self.settings.update(kwargs) #store all inputs, so they can be updated.

    def reinitialise(self, **kwargs):
        """
        Reinitialize the SharedDataCube part of this instance with new parameters.
        Only update parameters provided in kwargs; preserve those that are not changed.
        """
        self.settings.update(kwargs)
        OpenHSI.__init__(self, **self.settings)
        
    def __enter__(self):
        return self
    
    def __close__(self):
        self.stop_cam()

    def __exit__(self, exc_type, exc_value, traceback):
        self.stop_cam()
        
    def collect(self):
        """Collect the hyperspectral datacube."""
        self.start_cam()
        for i in tqdm(range(self.n_lines)):
            self.put(self.get_img())
            
            if callable(getattr(self,"get_temp",None)):
                self.cam_temperatures.put( self.get_temp() )
        self.stop_cam()
        
    def avgNimgs(self, n:int, # number of images to average
                ) -> np.ndarray: # averaged image
        """Take `n` images and find the average"""
        data = np.zeros(tuple(self.settings['resolution'])+(n,),np.int32)
        
        self.start_cam()
        for f in range(n):
            data[:,:,f]=self.get_img()
        self.stop_cam()
        return np.mean(data,axis=2)

In [ ]:
show_doc(OpenHSI.collect)

---

[source](https://github.com/openhsi/openhsi/blob/master/openhsi/capture.py#L55){target="_blank" style="float:right; font-size:smaller"}

### OpenHSI.collect

>      OpenHSI.collect ()

*Collect the hyperspectral datacube.*

In [ ]:
show_doc(OpenHSI.avgNimgs)

---

[source](https://github.com/openhsi/openhsi/blob/master/openhsi/capture.py#L65){target="_blank" style="float:right; font-size:smaller"}

### OpenHSI.avgNimgs

>      OpenHSI.avgNimgs (n:int)

*Take `n` images and find the average*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| n | int | number of images to average |
| **Returns** | **ndarray** | **averaged image** |

:::{.callout-warning}

Running in notebook slow downs the camera more than running in a script.

:::

To add a custom camera, five methods need to be defined in a class to:
1. Initialise camera `__init__`,  and
2. Open camera `start_cam`, and
3. Close camera `stop_cam`,  and
4. Capture a picture as a numpy array `get_img`, and
5. Update the exposure settings `set_exposure`, and
6. [Optional] Poll the camera temperature `get_temp`.

By inheriting from the `OpenHSI` class, all the methods to load settings/calibration files, collect datacube, saving data to NetCDF, and viewing as RGB are integrated. Furthermore, the custom camera class can be passed to a `SettingsBuilder` class for calibration. 

For example, we implement a simulated camera below. 

## Loading and processing datacubes further

:::{.callout-tip}

ProcessRawDatacube only works for raw data captured using `processing_lvl = -1`.

:::

In [ ]:
#| export

class ProcessRawDatacube(OpenHSI):
    """Post-process datacubes"""
    def __init__(self, fname:str, processing_lvl:int, json_path:str, cal_path:str, old_style:bool=False):
        """Post-process datacubes"""
        self.fname = fname
        self.buff = DataCube()
        self.buff.load_nc(fname, old_style=old_style)
        if hasattr(self.buff,"ds_temperatures"):
            self.get_temp = lambda: -999 # this function needs to exist to create temperature buffer
        super().__init__(n_lines=self.buff.dc.data.shape[1], processing_lvl=processing_lvl, json_path=json_path, cal_path=cal_path)
    
    def start_cam(self):
        pass
    
    def stop_cam(self):
        pass
    
    def get_img(self) -> np.ndarray:
        return self.buff.dc.get()
    
    def set_exposure(self):
        pass
    
    @delegates(OpenHSI.save)
    def save(self,save_dir:str, **kwargs):
        """Saves to a NetCDF file (and RGB representation) to directory dir_path in folder given by date with file name given by UTC time.
        Override the processing buffer timestamps with the timestamps in original file, also for camera temperatures."""
        # self.timestamps.data = self.buff.ds_timestamps
        self.timestamps.data = self.buff.timestamps.data
        if hasattr(self.buff,"ds_metadata"):
            self.ds_metadata = self.buff.ds_metadata
        if hasattr(self.buff,"ds_temperatures"):
            self.cam_temperatures.data = self.buff.ds_temperatures
        super().save(save_dir=save_dir, **kwargs)

In [ ]:
show_doc(ProcessRawDatacube.save)

---

[source](https://github.com/openhsi/openhsi/blob/master/openhsi/capture.py#L130){target="_blank" style="float:right; font-size:smaller"}

### ProcessRawDatacube.save

>      ProcessRawDatacube.save (save_dir:str, preconfig_meta_path:str=None,
>                               prefix:str='', suffix:str='',
>                               old_style:bool=False)

*Saves to a NetCDF file (and RGB representation) to directory dir_path in folder given by date with file name given by UTC time.
Override the processing buffer timestamps with the timestamps in original file, also for camera temperatures.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| save_dir | str |  | Path to folder where all datacubes will be saved at |
| preconfig_meta_path | str | None | Path to a .json file that includes metadata fields to be saved inside datacube |
| prefix | str |  | Prepend a custom prefix to your file name |
| suffix | str |  | Append a custom suffix to your file name |
| old_style | bool | False | Order of axis: True for (cross-track, along-track, wavelength), False for (wavelength, cross-track, along-track) |

```python
json_path = '../calibration_files/OpenHSI-16_settings_Mono8_bin2.json'
cal_path  = '../calibration_files/OpenHSI-16_calibration_Mono8_bin2_window.pkl'

proc_dc = ProcessRawDatacube(fname = "../../Downloads/16_pvn1_bin2_10ms2022_01_13-04_22_25.nc", processing_lvl=4,
                             json_path=json_path, cal_path=cal_path)
proc_dc.collect()

proc_dc.show(hist_eq=True)
```

If your saved datacubes have already been processed (for example, binned for smaller file size), you can further post-process your datacube using `ProcessDatacube`. A list of callable transforms can be provided to `ProcessDatacube.load_next_tfms`, the catch is to remember what transforms have already been applied during data collection and the final desired processing level (binning, radiance output, ...). See the [quick start guide](https://openhsi.github.io/openhsi/tutorial_camera.html) for some documentation on what is done for each processing level. 

:::{.callout-warning}

`next_tfms` needs to be valid. For instance, you cannot bin twice!

:::

In [ ]:
#| export

@delegates()
class ProcessDatacube(ProcessRawDatacube):
    """Post-process datacubes"""
    def __init__(self, fname:str, processing_lvl:int, json_path:str, cal_path:str, old_style:bool=False, **kwargs):
        """Post-process datacubes further!"""
        super().__init__(**kwargs)
    
    def load_next_tfms(self, next_tfms:List[Callable[[np.ndarray],np.ndarray]] = []):
        """provide the transforms you want to apply to this dataset"""
        self.tfm_list = next_tfms
        

In [ ]:
show_doc(ProcessDatacube.load_next_tfms)

---

[source](https://github.com/openhsi/openhsi/blob/master/openhsi/capture.py#L148){target="_blank" style="float:right; font-size:smaller"}

### ProcessDatacube.load_next_tfms

>      ProcessDatacube.load_next_tfms
>                                      (next_tfms:List[Callable[[numpy.ndarray],
>                                      numpy.ndarray]]=[])

*provide the transforms you want to apply to this dataset*

```python
proced_dc = ProcessDatacube(fname = "../calibration_files/2022_01_13/2022_01_13-04_22_25_proc_lvl_2.nc", processing_lvl=4,
                             json_path=json_path, cal_path=cal_path)
proced_dc.load_next_tfms([proced_dc.dn2rad])

proced_dc.collect()

proced_dc.show(hist_eq=True)
```

Due to requiring double the amount of memory and more to facilitate saving in a separate process, make sure your datacubes can fit in your RAM. Have not tested this but I would suggest choosing `n_lines` <= 1/3 the amount used using the regular OpenHSI. 

### The SimulatedCamera defintions are now part of the camera api.

In [ ]:
#| export
#| hide
import warnings

_DEPRECATED_CLASSES = {
    'SharedSimulatedCamera': 'openhsi.cameras',
    'SimulatedCamera': 'openhsi.cameras'
}

def __getattr__(name):
    if name in _DEPRECATED_CLASSES:
        new_module = _DEPRECATED_CLASSES[name]
        warnings.warn(
            f"Importing {name} from openhsi.capture is deprecated. "
            f"Please import from {new_module} instead. "
            "This compatibility may be removed in a future version.",
            DeprecationWarning,
            stacklevel=2
        )
        
        # Dynamic import
        from importlib import import_module
        module = import_module(new_module)
        return getattr(module, name)
    
    raise AttributeError(f"module '{__name__}' has no attribute '{name}'")

def __dir__():
    return list(_DEPRECATED_CLASSES.keys())

In [ ]:

#| hide
import nbdev; nbdev.nbdev_export()